In [1]:
from keras.applications import resnet50
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
# importing the weights of pre trained model resnet50
model=resnet50.ResNet50(weights='imagenet' ,input_shape=(224,224,3) , include_top=False)

E:\AnacondaPrompt\envs\mlopsnew\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
model.layers

In [4]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [5]:
# freezing all the layers of the model
for layers in model.layers:
    layers.trainable = False

In [6]:
model.layers[-1].output

<tf.Tensor 'activation_49/Relu:0' shape=(None, 7, 7, 2048) dtype=float32>

In [7]:
# adding layer to the pre trained model
t_layer=model.output
t_layer=Flatten()(t_layer)
t_layer=Dense(units=1024,activation='relu')(t_layer)
t_layer=Dense(units=1024,activation='relu')(t_layer)
t_layer=Dense(units=512,activation='relu')(t_layer)
t_layer=Dense(units=3,activation='softmax')(t_layer)

In [8]:
#merging the pre trained model with created layers
n_model=Model(inputs= model.input , outputs=t_layer)

In [9]:
n_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [10]:
n_model.output

<tf.Tensor 'dense_4/Softmax:0' shape=(None, 3) dtype=float32>

In [11]:
# making the augmented images from our data set
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        'dataset/train_set/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
test_set = test_datagen.flow_from_directory(
        'dataset/test_set/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 300 images belonging to 3 classes.
Found 108 images belonging to 3 classes.


In [12]:
n_model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])


In [13]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("face_detect.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Enter the number of training and validation samples here
nb_train_samples = 300
nb_validation_samples = 108

# We only train 5 EPOCHS 
epochs = 3
batch_size = 16

history = n_model.fit_generator(
    training_set,
    steps_per_epoch = nb_train_samples // batch_size ,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = test_set,
    validation_steps = nb_validation_samples // batch_size )

Epoch 1/3
18/18 [==============================] - 201s 11s/step - loss: 67.5454 - accuracy: 0.8116 - val_loss: 19.0665 - val_accuracy: 0.3081

Epoch 00001: val_loss improved from inf to 19.06648, saving model to face_detect.h5
Epoch 2/3
18/18 [==============================] - 195s 11s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 19.5427 - val_accuracy: 0.3618

Epoch 00002: val_loss did not improve from 19.06648
Epoch 3/3
18/18 [==============================] - 194s 11s/step - loss: 0.0783 - accuracy: 0.9981 - val_loss: 6.6970 - val_accuracy: 0.3023

Epoch 00003: val_loss improved from 19.06648 to 6.69696, saving model to face_detect.h5


In [14]:
# getting the index of the label
training_set.class_indices

{'n0': 0, 'n1': 1, 'n2': 2}

In [15]:
# importing the model which we have created
from keras.models import load_model
model=load_model('face_detect.h5')

In [ ]:
# predicting the images from the test data
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

pred_dict = {"[1]": "akshya ", 
             "[0]": "gaurav",
             "[2]": "papa",
            }

pred_dict_n = {"n0": "akshya", 
               "n1": "gaurav",
               "n2": "papa",
                       }

def draw_test(name, pred, im):
    face_dict = pred_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face_dict, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + pred_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("dataset/train_set/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(model.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - akshya
Class - akshya
Class - gaurav
Class - gaurav
Class - gaurav
Class - papa
Class - gaurav
Class - gaurav
